# события просмотра

In [1]:
#!scp jupyter-amtsu@62.109.12.193:/home/jupyter-amtsu/project/talent_stream/db.sqlite3 synchronize_data@94.250.254.32:/home/synchronize_data/hh_analytics/db.sqlite3 

In [2]:
!pwd

/home/jupyter-amtsu/github/team22/users/amtsurkov/analytics


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import sqlite3
import json
import seaborn

from datetime import datetime

# Сброс ограничений на количество выводимых рядов
pd.set_option('display.max_rows', None)
 
# Сброс ограничений на число столбцов
pd.set_option('display.max_columns', None)
 
# Сброс ограничений на количество символов в записи
pd.set_option('display.max_colwidth', None)

/tmp/ipykernel_2027611/3908925216.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
!whoami

jupyter-amtsu


In [5]:
connection = sqlite3.connect('/home/synchronize_data/hh_analytics/db.sqlite3')
cursor = connection.cursor()

In [6]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
cursor.fetchall()

[('django_migrations',),
 ('sqlite_sequence',),
 ('django_content_type',),
 ('auth_group_permissions',),
 ('auth_permission',),
 ('auth_group',),
 ('virtual_card_user_groups',),
 ('virtual_card_user_user_permissions',),
 ('django_admin_log',),
 ('django_session',),
 ('virtual_card_feedback',),
 ('virtual_card_user',),
 ('virtual_card_role',),
 ('virtual_card_contact',),
 ('virtual_card_event',)]

In [7]:
cursor.execute("SELECT sql FROM sqlite_master WHERE type='table' AND name='virtual_card_event'; ")
cursor.fetchall()

[('CREATE TABLE "virtual_card_event" ("id" integer NOT NULL PRIMARY KEY AUTOINCREMENT, "datetime_create" datetime NULL, "cookies" varchar(128) NULL, "values" text NULL CHECK ((JSON_VALID("values") OR "values" IS NULL)), "create_account_id" bigint NULL REFERENCES "virtual_card_user" ("id") DEFERRABLE INITIALLY DEFERRED)',)]

In [8]:
cursor.execute("""
SELECT         
            *
FROM virtual_card_event
where
    --"values" = NULL
    "values" is null or "values" = ''
    --ifnull("values", '') = ''
    --coalesce("values", '') = ''
    --ifnull(length("values"), 0) = 0
LIMIT 3
""")
cursor.fetchall()

[(1129, '2024-05-19 13:36:42.728787', '', None, 1),
 (1131, '2024-05-19 13:36:49.879086', '', None, 1),
 (1133, '2024-05-19 13:36:53.434405', '', None, 1)]

In [9]:
cursor.execute("""
SELECT 
            "values",
            count(1) as c
FROM virtual_card_event
where 
    not "values" is  null
    and not "values" = ''
group by "values"
order by "values" asc
LIMIT 3
""")
cursor.fetchall()

[('"1111"', 2),
 ('"{2:1}"', 1),
 ('{"event_type": "ask_not_allow_contact", "datetime": 1716279879391, "timezone": -180}',
  1)]

In [10]:
cursor.execute("""
SELECT 
            id,
            "datetime_create",
            "values",
            "create_account_id"
FROM virtual_card_event
where 
    not "values" is  null
    and not "values" = ''
    --'values' != "" 
    --and 'values' is not null
    --and length('values')> 1
LIMIT 1000000
""")

df = pd.DataFrame(cursor.fetchall(), columns=[
            "id",
            "datetime_create",
            "values",
            "create_account_id"
])

In [11]:
#df = df[100:]

In [12]:
#df = df[:30]

In [13]:
#df = df[:10]

In [14]:
df[:3]

,id,datetime_create,values,create_account_id
0,1,2024-05-16 12:52:47.643424,"""{2:1}""",1.0
1,2,2024-05-16 12:53:04.871949,"{""gg"": 11, ""hhh"": ""fff""}",1.0
2,3,2024-05-16 13:09:49.505042,"{""gg"": 11, ""hhh"": ""fff""}",1.0


In [15]:
df[1010:1012]

,id,datetime_create,values,create_account_id
1010,1011,2024-05-19 13:19:49.289464,"{""second"": 10, ""url"": ""https://learn.javascript.ru/settimeout-setinterval?fds"", ""event_type"": ""ping""}",1.0
1011,1012,2024-05-19 13:19:51.307119,"{""second"": 10, ""url"": ""https://www.liveinternet.ru/stat/andall/24hours.html?ready=32460"", ""event_type"": ""ping""}",1.0


In [16]:
#df["values"]

In [17]:
exploded_df = df.explode("values",ignore_index = True)

In [18]:
#exploded_df["values"]

In [19]:
exploded_df.describe()

,id,create_account_id
count,179681.000000,178562.000000
mean,89843.981150,10.139397
std,51869.613641,1.395949
min,1.000000,1.000000
25%,44924.000000,10.000000
50%,89844.000000,10.000000
75%,134764.000000,10.000000
max,179684.000000,16.000000


In [20]:
exploded_df.count()

id                   179681
datetime_create      179681
values               179681
create_account_id    178562
dtype: int64

In [21]:
exploded_df["values"] = exploded_df["values"].apply(json.loads)

In [22]:
x_df =  exploded_df["values"].apply(pd.Series)

In [23]:
#x_df = df.explode("values",ignore_index = True)

In [24]:
#x_df = x_df["values"].apply(json.loads)

In [25]:
#x_df1 =  x_df["values"].apply(pd.Series)

In [26]:
#x_df[:3]

In [27]:
#exploded_df['values'][5:7]

In [28]:
#exploded_df['values'][5:7].apply(pd.Series)

In [29]:
#df[:3]

In [30]:
#x_df[:3]

In [31]:
df = df.join(x_df)

In [32]:
df[10:13]

,id,datetime_create,values,create_account_id,0,gg,hhh,ff,url,title,event_type,url_1,referrer,head,ddd,second,datetime,timezone
10,11,2024-05-16 16:17:38.149086,"{""url"": ""https://talent.absrent.ru/admin/virtual_card/event/10/change/"", ""title"": ""Event object (10) | Change event | Django site admin"", ""event_type"": ""2open_extension""}",1.0,NaN,NaN,NaN,NaN,https://talent.absrent.ru/admin/virtual_card/event/10/change/,Event object (10) | Change event | Django site admin,2open_extension,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,12,2024-05-16 16:18:50.844970,"{""url"": ""https://talent.absrent.ru/admin/virtual_card/event/"", ""title"": ""Select event to change | Django site admin"", ""event_type"": ""open_extension""}",1.0,NaN,NaN,NaN,NaN,https://talent.absrent.ru/admin/virtual_card/event/,Select event to change | Django site admin,open_extension,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,13,2024-05-16 16:18:59.360631,"{""url"": ""https://talent.absrent.ru/admin/virtual_card/event/"", ""title"": ""Select event to change | Django site admin"", ""event_type"": ""open_extension""}",1.0,NaN,NaN,NaN,NaN,https://talent.absrent.ru/admin/virtual_card/event/,Select event to change | Django site admin,open_extension,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
#df.drop('column_name', axis=1)

In [34]:
#df.drop('gg', axis=1)

In [35]:
#df.drop( 'hhh', axis=1)

In [36]:
#df.drop( 'ff', axis=1)

In [37]:
#df[10:13]

In [38]:
df.count()

id                   179681
datetime_create      179681
values               179681
create_account_id    178562
0                         3
gg                        4
hhh                       3
ff                        1
url                  179404
title                  3298
event_type           179671
url_1                     3
referrer               2846
head                      2
ddd                       2
second               175993
datetime             178728
timezone             178304
dtype: int64

In [39]:
user_current_page_for_ping = {}
j = []
for i in df.itertuples():
    if i.second == 1:
        #if not user_current_page_for_ping.get(f'u{i.create_account_id}'):
        user_current_page_for_ping[f'u{i.create_account_id}'] = i.url
    pass
    if user_current_page_for_ping.get(f'u{i.create_account_id}', None) == i.url:
        #j.append(i)
        j.append((i.create_account_id, i.url, i.datetime_create, i.event_type, i.second, i.url.split('/')[2], datetime.strptime(i.datetime_create, '%Y-%m-%d %H:%M:%S.%f').date() ))



In [40]:
#datetime.strptime('2024-05-19 12:22:46.442592', '%Y-%m-%d %H:%M:%S.%f')

In [41]:
print(j[1141])

(8.0, 'https://www.tinkoff.ru/terminal/', '2024-05-20 07:20:43.715647', 'ping', 1.0, 'www.tinkoff.ru', datetime.date(2024, 5, 20))


In [42]:
len(j)

30645

In [43]:

df2 = pd.DataFrame(j, columns=[
    'create_account_id',
    'url', 
    'datetime_create', 
    'event_type',
    'second',
    'domain',
    'date'
])


df2 = pd.DataFrame(j, columns=[
    'Index11',
    'id=3153', 
    'datetime_create',
    'values',
    'create_account_id',
    '_5',
    'gg',
    'hhh',
    'ff',
    'url', 
    'title',
    'event_type',
    'url_1',
    'referrer',
    'head',
    'ddd',
    'second',
    'datetime',
    'timezone'

            #"id",
           # "datetime_create",
           # "values",
            #"create_account_id"
])

In [44]:
df2[:10]


,create_account_id,url,datetime_create,event_type,second,domain,date
0,1.0,https://learn.javascript.ru/call-apply-decorators,2024-05-19 12:22:46.442592,ping,1.0,learn.javascript.ru,2024-05-19
1,1.0,https://learn.javascript.ru/settimeout-setinterval?fds,2024-05-19 12:22:48.626797,ping,1.0,learn.javascript.ru,2024-05-19
2,1.0,https://learn.javascript.ru/settimeout-setinterval?fds,2024-05-19 12:22:50.628702,ping,2.0,learn.javascript.ru,2024-05-19
3,1.0,https://learn.javascript.ru/settimeout-setinterval?fds,2024-05-19 12:22:55.633894,ping,5.0,learn.javascript.ru,2024-05-19
4,1.0,https://learn.javascript.ru/settimeout-setinterval?fds,2024-05-19 12:23:15.638529,ping,10.0,learn.javascript.ru,2024-05-19
5,1.0,https://learn.javascript.ru/settimeout-setinterval?fds,2024-05-19 12:23:25.642206,ping,10.0,learn.javascript.ru,2024-05-19
6,1.0,https://learn.javascript.ru/settimeout-setinterval?fds,2024-05-19 12:23:35.640899,ping,10.0,learn.javascript.ru,2024-05-19
7,1.0,https://learn.javascript.ru/settimeout-setinterval?fds,2024-05-19 12:23:45.635846,ping,10.0,learn.javascript.ru,2024-05-19
8,1.0,https://learn.javascript.ru/settimeout-setinterval?fds,2024-05-19 12:23:56.373527,ping,10.0,learn.javascript.ru,2024-05-19
9,1.0,https://www.liveinternet.ru/stat/andall/24hours.html?ready=32460,2024-05-19 12:24:04.168558,ping,1.0,www.liveinternet.ru,2024-05-19


In [45]:
df2.groupby(['create_account_id', 'date']).mean('second')

second
create_account_id date                 
1.0               2024-05-19  16.976501
2.0               2024-05-19  11.041667
8.0               2024-05-20   4.742222
9.0               2024-05-20  35.528249
                  2024-05-21  16.512301
10.0              2024-05-21  31.115489
                  2024-05-22  23.069052
                  2024-05-24  28.815658
                  2024-05-25  12.123457
                  2024-05-26  22.831409
                  2024-05-27   6.670445
                  2024-05-28  52.306769
                  2024-05-29  49.626582
                  2024-05-30  30.633245
                  2024-05-31  27.908759
                  2024-06-01  14.304598
                  2024-06-02   9.186207
11.0              2024-05-22   5.571429
                  2024-05-25  44.382353
12.0              2024-05-22  50.817073
                  2024-05-23  16.740698
                  2024-05-24  15.883721
                  2024-05-25  30.868914
                  2024-05-26  32.538776
                  2024-05-27  20.726087
                  2024-05-28  16.791332
                  2024-05-29  14.819209
                  2024-05-30   9.327381
                  2024-05-31  17.676040
15.0              2024-05-25   8.722222
16.0              2024-05-25  39.655844

In [46]:
df2.groupby(['create_account_id', 'date']).sum('second')

second
create_account_id date                
1.0               2024-05-19   13004.0
2.0               2024-05-19     530.0
8.0               2024-05-20    2134.0
9.0               2024-05-20  115076.0
                  2024-05-21   22820.0
10.0              2024-05-21  103459.0
                  2024-05-22   36749.0
                  2024-05-24   74719.0
                  2024-05-25    3928.0
                  2024-05-26   29658.0
                  2024-05-27    8238.0
                  2024-05-28   47913.0
                  2024-05-29   39205.0
                  2024-05-30   11610.0
                  2024-05-31    7647.0
                  2024-06-01   17423.0
                  2024-06-02    3996.0
11.0              2024-05-22      39.0
                  2024-05-25    1509.0
12.0              2024-05-22    8334.0
                  2024-05-23   29246.0
                  2024-05-24   19807.0
                  2024-05-25    8242.0
                  2024-05-26    7972.0
                  2024-05-27   19068.0
                  2024-05-28   20922.0
                  2024-05-29   28853.0
                  2024-05-30    4701.0
                  2024-05-31   15714.0
15.0              2024-05-25     157.0
16.0              2024-05-25    6107.0

In [47]:
df2.loc[df2['create_account_id'] == 10].groupby(['date', 'domain']).sum('second').sort_values(by=['date','second'], ascending=False)

create_account_id    second
date       domain                                                     
2024-06-02 trudvsem.ru                                 440.0    1365.0
           www.liveinternet.ru                         280.0    1211.0
           www.ozon.ru                                 970.0     427.0
           vc.ru                                       230.0     394.0
           www.rusklimat.ru                            150.0      92.0
           level.ru                                    200.0      75.0
           hh.ru                                       250.0      60.0
           www.avito.ru                                160.0      34.0
           gdeposylka.ru                               120.0      28.0
           rabotka.com.ru                               90.0      27.0
           www.ktgs.ru                                 100.0      27.0
           dzen.ru                                     100.0      22.0
           www.careerjet.ru                             70.0      17.0
           aflink.ru                                    50.0      15.0
           www.superjob.ru                              80.0      12.0
           geekjob.ru                                   50.0      11.0
           gorodrabot.ru                                50.0      11.0
           leroymerlin.ru                               60.0      11.0
           r21.spb.ru                                   50.0      11.0
           rdw.ru                                       50.0      11.0
           www.rabota.ru                                50.0      11.0
           jobkadrov.ru                                 40.0       9.0
           joblab.ru                                    40.0       9.0
           tproger.ru                                   40.0       9.0
           um-climat.ru                                 40.0       9.0
           www.zarplata.ru                              30.0       8.0
           www.dns-shop.ru                              50.0       7.0
           ca-em.ru                                     20.0       6.0
           careerist.ru                                 40.0       6.0
           fut.ru                                       40.0       6.0
           rb.ru                                        40.0       6.0
           promelvent.ru                                30.0       4.0
           tiei.ru                                      30.0       4.0
           www.wiweb.ru                                 30.0       4.0
           yandex.ru                                    30.0       4.0
           1000dosok.ru                                 20.0       3.0
           4brain.ru                                    20.0       3.0
           nsuem.ru                                     20.0       3.0
           profi.ru                                     30.0       3.0
           www.bigam.ru                                 20.0       3.0
           www.terdo.ru                                 20.0       3.0
           career.hh.ru                                 20.0       2.0
           lifehacker.ru                                20.0       2.0
           market.yandex.ru                             20.0       2.0
           www.mvideo.ru                                20.0       2.0
           www.pndtech.ru                               20.0       2.0
           www.vseinstrumenti.ru                        20.0       2.0
           ekdo.trudvsem.ru                             10.0       1.0
           soware.ru                                    10.0       1.0
           youla.ru                                     10.0       1.0
2024-06-01 blog.skillfactory.ru                       1820.0    9648.0
           www.ozon.ru                                7830.0    3930.0
           www.liveinternet.ru                         570.0    2374.0
           mircli.ru                                   550.0     738.0
           esia.gosuslugi.ru                           530.0     25

In [48]:
df2.groupby(['create_account_id', 'date', 'domain']).sum('second').sort_values(by=['create_account_id', 'date','second'], ascending=False)

second
create_account_id date       domain                                          
16.0              2024-05-25 talent.absrent.ru                         6107.0
15.0              2024-05-25 talent.absrent.ru                          155.0
                             hh.ru                                        1.0
                             trial-sport.ru                               1.0
12.0              2024-05-31 hh.ru                                    15711.0
                             csd.ru                                       3.0
                  2024-05-30 hh.ru                                     4701.0
                  2024-05-29 hh.ru                                    28797.0
                             www.tadviser.ru                             45.0
                             digital.cap.ru                              11.0
                  2024-05-28 hh.ru                                    19718.0
                             digital.cap.ru                            1080.0
                             taxcom.ru                                   68.0
                             pro.rabota.ru                               48.0
                             brandanalytics.ru                            8.0
                  2024-05-27 hh.ru                                    10145.0
                             vc.ru                                     8899.0
                             aeroidea.ru                                  8.0
                             buroburo.ru                                  8.0
                             practicum.yandex.ru                          8.0
                  2024-05-26 vc.ru                                     7300.0
                             hh.ru                                      444.0
                             news.pressfeed.ru                          228.0
                  2024-05-25 hh.ru                                     4668.0
                             vc.ru                                     3574.0
                  2024-05-24 hh.ru                                    19789.0
                             planetcalc.ru                               18.0
                  2024-05-23 hh.ru                                    29245.0
                             jazz.sberbank.ru                             1.0
                  2024-05-22 hh.ru                                     8328.0
                             talent.absrent.ru                            6.0
11.0              2024-05-25 talent.absrent.ru                         1509.0
                  2024-05-22 confluence.sberbank.ru                      38.0
                             talent.absrent.ru                            1.0
10.0              2024-06-02 trudvsem.ru                               1365.0
                             www.liveinternet.ru                       1211.0
                             www.ozon.ru                                427.0
                             vc.ru                                      394.0
                             www.rusklimat.ru                            92.0
                             level.ru                                    75.0
                             hh.ru                                       60.0
                             www.avito.ru                                34.0
                             gdeposylka.ru                               28.0
                             rabotka.com.ru                              27.0
                             www.ktgs.ru                                 27.0
                             dzen.ru                                     22.0
                             www.careerjet.ru                            17.0
                             aflink.ru                                   15.0
                             www.superjob.ru                             12.0
                             geekjob.ru                                  11.0
               

In [49]:
1320.0/60/60

0.36666666666666664

In [50]:
df2.loc[df2['create_account_id'] == 12.0].groupby(['date', 'domain']).sum('second').sort_values(by=['date','second'], ascending=False)

create_account_id   second
date       domain                                         
2024-05-31 hh.ru                          10644.0  15711.0
           csd.ru                            24.0      3.0
2024-05-30 hh.ru                           6312.0   4701.0
2024-05-29 hh.ru                          23172.0  28797.0
           www.tadviser.ru                  132.0     45.0
           digital.cap.ru                    60.0     11.0
2024-05-28 hh.ru                          14328.0  19718.0
           digital.cap.ru                   600.0   1080.0
           taxcom.ru                        108.0     68.0
           pro.rabota.ru                     84.0     48.0
           brandanalytics.ru                 36.0      8.0
2024-05-27 hh.ru                           7104.0  10145.0
           vc.ru                           3864.0   8899.0
           aeroidea.ru                       36.0      8.0
           buroburo.ru                       36.0      8.0
           practicum.yandex.ru               36.0      8.0
2024-05-26 vc.ru                           2340.0   7300.0
           hh.ru                            312.0    444.0
           news.pressfeed.ru                300.0    228.0
2024-05-25 hh.ru                           1920.0   4668.0
           vc.ru                           1368.0   3574.0
2024-05-24 hh.ru                          15012.0  19789.0
           planetcalc.ru                     48.0     18.0
2024-05-23 hh.ru                          21036.0  29245.0
           jazz.sberbank.ru                  12.0      1.0
2024-05-22 hh.ru                           1920.0   8328.0
           talent.absrent.ru                 48.0      6.0

In [51]:
df2.loc[df2['create_account_id'] == 12.0].groupby(['url']).sum('second').sort_values(by=['second'], ascending=False)

,create_account_id,second
url,,
https://hh.ru/search/resume?text=%D0%BF%D1%80%D0%BE%D0%B4%D0%B6%D0%B5%D0%BA%D1%82+or+%22%D0%BF%D1%80%D0%BE%D0%B5%D0%BA%D1%82%D0%BD%D1%8B%D0%B9+%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80%22+or+%22project+manager%22+or+%22%D1%80%D1%83%D0%BA%D0%BE%D0%B2%D0%BE%D0%B4%D0%B8%D1%82%D0%B5%D0%BB%D1%8C+%D0%BF%D1%80%D0%BE%D0%B5%D0%BA%D1%82%D0%BE%D0%B2%22+or+%22%D1%80%D1%83%D0%BA%D0%BE%D0%B2%D0%BE%D0%B4%D0%B8%D1%82%D0%B5%D0%BB%D1%8C+%D0%BF%D1%80%D0%BE%D0%B5%D0%BA%D1%82%D0%B0%22+or+%22lead+progect%22&text=senior+or+lead+or+grouphead+or+%27group+head%27&text=%D1%80%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%BA%D0%B0+or+%D1%80%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%87%D0%B8%D0%BA+or+web+or+it+or+%D0%98%D0%A2&text=%D1%81%D0%B0%D0%BC%D0%BE%D0%BF%D0%B8%D1%81%D0%BD%D1%8B%D0%B9+or+%D1%81%D0%B0%D0%BC%D0%BE%D0%BF%D0%B8%D1%81%D0%BD%D1%8B%D0%B5+or+%D0%BA%D0%B0%D1%81%D1%82%D0%BE%D0%BC%D0%BD%D1%8B%D0%B9+or+%D0%BA%D0%B0%D1%81%D1%82%D0%BE%D0%BC%D0%BD%D1%8B%D0%B5&logic=normal&logic=normal&logic=normal&logic=normal&pos=position%2Cworkplace_position&pos=workplace_position%2Cposition&pos=full_text&pos=full_text&exp_period=all_time&exp_period=all_time&exp_period=all_time&exp_period=all_time&exp_company_size=any&exp_company_size=any&exp_company_size=any&exp_company_size=any&show_conditions=false&filter_exp_period=all_time&area=1&area=2019&relocation=living_or_relocation&age_from=&age_to=&gender=unknown&salary_from=&salary_to=300000&currency_code=RUR&label=exclude_viewed_by_user_id&order_by=relevance&search_period=30&items_on_page=50&no_magic=true&job_search_status_changed_by_user=true&page=1,4452.0,19011.0
https://hh.ru/search/resume?text=%D0%BF%D1%80%D0%BE%D0%B4%D0%B6%D0%B5%D0%BA%D1%82+or+%22%D0%BF%D1%80%D0%BE%D0%B5%D0%BA%D1%82%D0%BD%D1%8B%D0%B9+%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80%22+or+%22project+manager%22+or+%22%D1%80%D1%83%D0%BA%D0%BE%D0%B2%D0%BE%D0%B4%D0%B8%D1%82%D0%B5%D0%BB%D1%8C+%D0%BF%D1%80%D0%BE%D0%B5%D0%BA%D1%82%D0%BE%D0%B2%22+or+%22%D1%80%D1%83%D0%BA%D0%BE%D0%B2%D0%BE%D0%B4%D0%B8%D1%82%D0%B5%D0%BB%D1%8C+%D0%BF%D1%80%D0%BE%D0%B5%D0%BA%D1%82%D0%B0%22+or+%22%D0%BF%D1%80%D0%BE%D0%B4%D1%83%D0%BA%D1%82+%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80%22+or+%22%D0%BF%D1%80%D0%BE%D0%B4%D1%83%D0%BA%D1%82%D0%BE%D0%B2%D1%8B%D0%B9+%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80%22+or+%22%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80+%D0%BF%D1%80%D0%BE%D0%B4%D1%83%D0%BA%D1%82%D0%B0%22+or+%22%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80+%D0%BF%D1%80%D0%BE%D0%B4%D1%83%D0%BA%D1%82%D0%BE%D0%B2%22&text=%D0%90%D0%B3%D0%B5%D0%BD%D1%82%D1%81%D1%82%D0%B2%D0%BE&text=senior+or+lead+or+grouphead+or+%27group+head%27&text=%D1%80%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%BA%D0%B0+or+%D1%80%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%87%D0%B8%D0%BA+or+web+or+it&text=%D1%81%D0%BC%D0%B8+or+%D0%BC%D0%B0%D1%80%D0%BA%D0%B5%D1%82%D0%B8%D0%BD%D0%B3&logic=normal&logic=normal&logic=normal&logic=normal&logic=normal&pos=position%2Cworkplace_position&pos=workplace_organization&pos=workplace_position%2Cposition&pos=full_text&pos=workplace_organization&exp_period=all_time&exp_period=last_year&exp_period=all_time&exp_period=all_time&exp_period=last_three_years&exp_company_size=any&exp_company_size=any&exp_company_size=any&exp_company_size=any&exp_company_size=any&show_conditions=false&filter_exp_period=all_time&area=1&area=2019&relocation=living_or_relocation&age_from=&age_to=&gender=unknown&salary_from=&salary_to=&currency_code=RUR&label=exclude_viewed_by_user_id&order_by=relevance&search_period=365&items_on_page=50&no_magic=true&page=3&disableBrowserCache=true,4596.0,14320.0
https://vc.ru/hr/1037767-kak-v-ekonive-apk-avtomatizirovali-podbor-i-osvobodili-rekruterov-ot-rutiny,3864.0,13076.0
https://hh.ru/search/resume?text=%D0%BF%D1%80%D0%BE%D0%B4%D0%B6%D0%B5%D0%BA%D1%82+or+%22%D0%BF%D1%80%D0%BE%D0%B5%D0%BA%D1%82%D0%BD%D1%8B%D0%B9+%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80%22+or+%22project+manager%22+or+%22%D1%80%D1%83

In [52]:
#df.loc[df['column_name'] == some_value]

In [53]:
65629/60/60

18.230277777777776